In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn import ensemble
from scipy.sparse import coo_matrix
from sklearn import svm

import xgboost
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

In [3]:
def w2v_average_wiki():
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))
    songs=utils.w2v_average_wiki(songs,'glove-wiki-gigaword-300')
    return np.array(songs),np.array(labels)

In [4]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [5]:
X,y=w2v_average_wiki()

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)

In [7]:
def dummy(X_train,X_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(X_train, y_train)
    y_pred=dummy.predict(X_test)
    return f1_score(y_test,y_pred)

In [27]:
print(dummy(X_train,X_test,y_train,y_test))

0.4434753438443475


In [9]:
def logistics_sentiment():
    conn=sqlite3.connect('SONGS.db')
    data=pd.read_sql("select Economic,neg,pos,neu,comp from song",conn).dropna()
    X=data[['neg','pos','neu','comp']]
    y=data.Economic
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [10]:
print(logistics_sentiment())

0.0


C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
def logistics(X_train,X_test,y_train,y_test):
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [12]:
print(logistics(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.47854657904909165


In [13]:
def naive_bayes(X_train,X_test,y_train,y_test):
    nb=GaussianNB()
    nb.fit(X_train, y_train)
    y_pred=nb.predict(X_test)
    return f1_score(y_test,y_pred)

In [14]:
print(naive_bayes(X_train,X_test,y_train,y_test))

0.5689223057644109


In [15]:
def support_vector(X_train,X_test,y_train,y_test):
    model = svm.SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [16]:
print(support_vector(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.0


C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
def knn(X_train,X_test,y_train,y_test):
    model = KNeighborsClassifier(n_neighbors=5) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [18]:
print(knn(X_train,X_test,y_train,y_test))

0.5415061295971979


In [19]:
def decision_tree(X_train,X_test,y_train,y_test):
    model = DecisionTreeClassifier(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [20]:
print(decision_tree(X_train,X_test,y_train,y_test))

0.566219470244238


In [21]:
def random_forest(X_train,X_test,y_train,y_test):
    model=ensemble.RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [22]:
print(random_forest(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5162569389373513


In [23]:
def xgb(X_train,X_test,y_train,y_test):
    X_train=coo_matrix(X_train).tocsc()
    X_test=coo_matrix(X_test).tocsc()
    model=xgboost.XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [24]:
print(xgb(X_train,X_test,y_train,y_test))

0.5545657015590201


In [25]:
def simple_nn(X_train,X_test,y_train,y_test):

    def create_model_architecture(input_size):
        input_layer = layers.Input(shape=(input_size,),)
        hidden_layer = layers.Dense(200, activation="relu")(input_layer)
        output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)
        classifier = models.Model(inputs = input_layer, outputs = output_layer)
        classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
        return classifier

    model=create_model_architecture(len(X_train[0]))
    model.fit(X_train, y_train,epochs=100)
    y_pred = [int(round(i[0])) for i in model.predict(X_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [26]:
print(simple_nn(X_train,X_test,y_train,y_test))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
3327/3327 [==============================] - 3s 957us/step - loss: 0.6837
Epoch 2/100
3327/3327 [==============================] - 0s 75us/step - loss: 0.6660
Epoch 3/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.6546
Epoch 4/100
3327/3327 [==============================] - 0s 71us/step - loss: 0.6426
Epoch 5/100
3327/3327 [==============================] - 0s 75us/step - loss: 0.6262
Epoch 6/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.6123
Epoch 7/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.5980
Epoch 8/100
3327/3327 [==============================] - 0s 71us/step - loss: 0.5788
Epoch 9/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.5629
Epoch 10/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.5450
Epoch 11/100
3327/3327 [====

3327/3327 [==============================] - 0s 80us/step - loss: 0.0595
Epoch 90/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0517
Epoch 91/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0525
Epoch 92/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0517
Epoch 93/100
3327/3327 [==============================] - 0s 80us/step - loss: 0.0551
Epoch 94/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0506
Epoch 95/100
3327/3327 [==============================] - 0s 66us/step - loss: 0.0500
Epoch 96/100
3327/3327 [==============================] - 0s 71us/step - loss: 0.0486
Epoch 97/100
3327/3327 [==============================] - 0s 66us/step - loss: 0.0532
Epoch 98/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0579
Epoch 99/100
3327/3327 [==============================] - 0s 75us/step - loss: 0.0747
Epoch 100/100
3327/3327 [==============================] - 0s 75us/